In [3]:
# Required imports
import tkinter as tk
from tkinter import ttk
from tkinter import StringVar, Listbox, MULTIPLE
import requests
import pandas as pd
from datetime import datetime
from itertools import product

# List of known bookmakers (Replace this with your own list or fetch dynamically)
known_bookmakers = ['FanDuel', 'BetUS', 'DraftKings', 'SuperBook', 'Caesars', 'LowVig.ag', 'BetOnline.ag', 'Bovada', 'TwinSpires', 'BetRivers', 'Unibet', 'MyBookie.ag', 'PointsBet (US)', 'WynnBET', 'BetMGM', 'Barstool Sportsbook']

# Initialize global variables for API quota
api_requests_used = 0
api_requests_remaining = 0

# Fetch and prepare odds data
def download_odds(sport, api_quota_label, selected_bookmakers='All'):
    global api_requests_used, api_requests_remaining
    
    url = f"https://api.the-odds-api.com/v4/sports/{sport}/odds/?apiKey=2c6684b5eb487d4e9e90bfbbffc2d903&regions=us&markets=h2h,spreads&oddsFormat=american"   #Backup API Key: 2c6684b5eb487d4e9e90bfbbffc2d903  / 0a961c52965144f19df17552e65f101e
    response = requests.get(url)
    
    api_requests_used = int(response.headers.get('x-requests-used', '0'))
    api_requests_remaining = int(response.headers.get('x-requests-remaining', '0'))
    
    # Update the API quota label
    api_quota_label.config(text=f"API Quota: {api_requests_used} used, {api_requests_remaining} remaining")
    
    result = response.json()
    all_matches = []

    for res in result:
        for books in res['bookmakers']:
            if selected_bookmakers != 'All' and books['title'] not in selected_bookmakers:
                continue
            match = {
                'event_time': datetime.fromisoformat(res['commence_time'].replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S'),
                'bookmaker': books['title'],
                'Fighter': books['markets'][0]['outcomes'][0]['name'],
                'Opponent': books['markets'][0]['outcomes'][1]['name'],
                'odds_f1': books['markets'][0]['outcomes'][0]['price'],
                'odds_f2': books['markets'][0]['outcomes'][1]['price']
            }
            all_matches.append(match)

    return pd.DataFrame(all_matches)


# Function to calculate arbitrage opportunities
def calculate_arbitrage(df):
    arb_rows = []

    for (event_time, Fighter, Opponent), event_df in df.groupby(['event_time', 'Fighter', 'Opponent']):
        max_arb_percent = -1  # Initialize with a value that will always be less than any arb_percent
        best_arb_row = None  # Initialize with None
        
        combinations = product(event_df.itertuples(index=False), repeat=2)
        
        for row1, row2 in combinations:
            if row1.bookmaker == row2.bookmaker:
                continue
            arb_percent = round((1 / row1.odds_f1 + 1 / row2.odds_f2) * 100, 2)
            
            # Only update if this combination gives a better (i.e., smaller) arb_percent
            if arb_percent < 100 and arb_percent > max_arb_percent:
                max_arb_percent = arb_percent
                best_arb_row = {
                    'arb_percent': arb_percent,
                    'event_time': event_time,
                    'Fighter': Fighter,
                    'odds_f1': row1.odds_f1,
                    'Opponent': Opponent,
                    'odds_f2': row2.odds_f2,
                    'bookmaker_f1': row1.bookmaker,
                    'bookmaker_f2': row2.bookmaker
                }
        
        if best_arb_row:
            arb_rows.append(best_arb_row)
                
    return pd.DataFrame(arb_rows)

# Function to populate the Tkinter table
def populate_table(tree, df):
    # Clear existing rows
    for row in tree.get_children():
        tree.delete(row)
    
    # Insert new rows
    for index, row in df.iterrows():
        tree.insert('', 'end', values=(row['arb_percent'], row['event_time'], row['Fighter'], row['odds_f1'], row['bookmaker_f1'], row['Opponent'], row['odds_f2'], row['bookmaker_f2']))

# Function to refresh the data and table
def refresh_table(tree, sport, api_quota_label, bookmaker_listbox):
    selected_indices = bookmaker_listbox.curselection()
    selected_bookmakers = [bookmaker_listbox.get(i) for i in selected_indices]
    
    # If "All" is selected, use 'All' to ignore the bookmaker filter
    if "All" in selected_bookmakers:
        selected_bookmakers = 'All'
    
    df = download_odds(sport, api_quota_label, selected_bookmakers)
    arb_df = calculate_arbitrage(df)
    populate_table(tree, arb_df)


# Create a mapping from human-readable names to sport keys
sport_name_mapping = {
    'UFC': 'mma_mixed_martial_arts',
    'BOXING': 'boxing_boxing',
    'NFL': 'americanfootball_nfl',
    'NCAAF': 'americanfootball_ncaaf',
    'MLB': 'baseball_mlb',
    'NBA': 'basketball_nba',
    'NHL': 'icehockey_nhl'
}

# Function to create a Treeview for a given sport
def create_sport_tab(tab_control, sport_key, human_readable_name, api_quota_label, auto_refresh_time=0):
    tab = ttk.Frame(tab_control)
    tab_control.add(tab, text=human_readable_name)  # Use the human-readable name here
    
    # Create Listbox for bookmakers
    bookmaker_listbox = Listbox(tab, selectmode=MULTIPLE)
    bookmaker_listbox.pack(side='right')
    
    # Add an "All" option along with known bookmakers
    bookmaker_listbox.insert(tk.END, "All")
    for bm in known_bookmakers:
        bookmaker_listbox.insert(tk.END, bm)
    
    # Column headers and Treeview
    columns = ('Arbitrage %', 'Event Time', 'Player', 'Best Odds P1', 'Bookmaker P1', 'Opponent', 'Best Odds P2', 'Bookmaker P2')
    tree = ttk.Treeview(tab, columns=columns, show='headings')
    
    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=100)

    tree.pack(side='left', fill='both', expand=True)
    
    # Scrollbar
    scrollbar = ttk.Scrollbar(tab, orient='vertical', command=tree.yview)
    scrollbar.pack(side='right', fill='y')
    tree.configure(yscrollcommand=scrollbar.set)
    
    # Modify the button for manual updates
    btn_refresh = ttk.Button(tab, text="Manual Refresh", command=lambda: refresh_table(tree, sport_key, api_quota_label, bookmaker_listbox))
    btn_refresh.pack(side='bottom')

    # Function to refresh the table periodically (if auto_refresh_time is set)
    def periodic_refresh():
        refresh_table(tree, sport_key, api_quota_label)
        root.after(auto_refresh_time, periodic_refresh)

    # Start periodic refresh if auto_refresh_time is set
    if auto_refresh_time:
        periodic_refresh()

# Tkinter GUI setup
root = tk.Tk()
root.title("Live Arbitrage Opportunities")

# Label to display API quota
api_quota_label = ttk.Label(root, text=f"API Quota: {api_requests_used} used, {api_requests_remaining} remaining")
api_quota_label.pack(side='bottom')

# Create a tab control
tab_control = ttk.Notebook(root)
tab_control.pack(expand=1, fill='both')

# Create tabs dynamically
for human_readable_name, sport_key in sport_name_mapping.items():
    create_sport_tab(tab_control, sport_key, human_readable_name, api_quota_label, auto_refresh_time=0)  # Set auto_refresh_time to 0 for no auto-refresh

# Tkinter event loop
root.mainloop()




Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\kelly\anaconda3\envs\TheSauce\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\kelly\AppData\Local\Temp\ipykernel_25964\2526030638.py", line 149, in <lambda>
    btn_refresh = ttk.Button(tab, text="Manual Refresh", command=lambda: refresh_table(tree, sport_key, api_quota_label, bookmaker_listbox))
                                                                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\kelly\AppData\Local\Temp\ipykernel_25964\2526030638.py", line 104, in refresh_table
    arb_df = calculate_arbitrage(df)
             ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\kelly\AppData\Local\Temp\ipykernel_25964\2526030638.py", line 54, in calculate_arbitrage
    for (event_time, Fighter, Opponent), event_df in df.groupby(['event_time', 'Fighter', 'Opponent']):
                     